In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

In [2]:
# import population csv

population_csv = "Resources/country_population.csv"
population_df = pd.read_csv(population_csv)
population_df.head()

,country,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,647500,"48,0","0,00","23,06","163,07",700.0,"36,0","3,2","12,13","0,22","87,65",1,"46,6","20,34","0,38","0,24","0,38"
1,Albania,EASTERN EUROPE,3581655,28748,"124,6","1,26","-4,93","21,52",4500.0,"86,5","71,2","21,09","4,42","74,49",3,"15,11","5,22","0,232","0,188","0,579"
2,Algeria,NORTHERN AFRICA,32930091,2381740,"13,8","0,04","-0,39",31,6000.0,"70,0","78,1","3,22","0,25","96,53",1,"17,14","4,61","0,101","0,6","0,298"
3,American Samoa,OCEANIA,57794,199,"290,4","58,29","-20,71","9,27",8000.0,"97,0","259,5",10,15,75,2,"22,46","3,27",NaN,NaN,NaN
4,Andorra,WESTERN EUROPE,71201,468,"152,1","0,00","6,6","4,05",19000.0,"100,0","497,2","2,22",0,"97,78",3,"8,71","6,25",NaN,NaN,NaN


In [3]:
# import alcohol csv
alcohol_csv = "Resources/country_alcohol.csv"
alcohol_df = pd.read_csv(alcohol_csv)
alcohol_df.head()

,country,alc,years
0,Afghanistan,0.000000,61.25205
1,Albania,4.879627,74.13000
2,Algeria,0.692020,71.82955
3,Andorra,12.332149,82.65301
4,Angola,5.660368,51.74093


In [4]:
#create new population df with desired columns

new_population_df = population_df[['country', 'Population', 'GDP ($ per capita)']].copy()
new_population_df.head()

,country,Population,GDP ($ per capita)
0,Afghanistan,31056997,700.0
1,Albania,3581655,4500.0
2,Algeria,32930091,6000.0
3,American Samoa,57794,8000.0
4,Andorra,71201,19000.0


In [5]:
# checking count for population
new_population_df.count()

country               227
Population            227
GDP ($ per capita)    226
dtype: int64

In [6]:
# Dropping null or value = 0
new_population_df = new_population_df.replace(0, np.nan)
new_population_df = new_population_df.dropna(axis = 0, how='any')
new_population_df.count()

country               226
Population            226
GDP ($ per capita)    226
dtype: int64

In [7]:
#rename columns for population_df

population_cols = ["country", "Population", "GDP ($ per capita)"]

population_transformed = new_population_df[population_cols].copy()

population_transformed = population_transformed.rename(columns={"country": "country",
                                                         "Population": "population",
                                                         "GDP ($ per capita)": "gdp"})

population_transformed.head()

,country,population,gdp
0,Afghanistan,31056997,700.0
1,Albania,3581655,4500.0
2,Algeria,32930091,6000.0
3,American Samoa,57794,8000.0
4,Andorra,71201,19000.0


In [8]:
# checking count for alcohol
alcohol_df.count()


country    193
alc        193
years      193
dtype: int64

In [9]:
# Dropping null or value = 0
alcohol_df = alcohol_df.replace(0, np.nan)
alcohol_df = alcohol_df.dropna (axis = 0, how='any')
alcohol_df.count()

country    180
alc        180
years      180
dtype: int64

In [10]:
# rename columns for alcohol_df

alcohol_cols = ["country", "alc", "years"]

alcohol_transformed = alcohol_df[alcohol_cols].copy()

alcohol_transformed = alcohol_transformed.rename(columns = {"country": "country",
                                                           "alc": "alcohol_consumption",
                                                           "years": "life_expectancy"})

alcohol_transformed.head()

,country,alcohol_consumption,life_expectancy
1,Albania,4.879627,74.13000
2,Algeria,0.692020,71.82955
3,Andorra,12.332149,82.65301
4,Angola,5.660368,51.74093
5,Antigua and Barbuda,4.879627,75.37800


In [11]:
#connect to Postgres

rds_connection_string = "postgres:0818@localhost:5432/ETL"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [12]:
engine.table_names()

[]

In [13]:
#load tables into Postgres

alcohol_transformed.to_sql(name='alcohol', con=engine, if_exists='append', index=False)

In [14]:
population_transformed.to_sql(name='population', con=engine, if_exists='append', index=False)

In [15]:
#check if tables have been updated

engine.table_names()

['alcohol', 'population']

In [16]:
#check for data within the tables

pd.read_sql_query('select * from alcohol', con=engine).head()

,country,alcohol_consumption,life_expectancy
0,Albania,4.879627,74.13000
1,Algeria,0.692020,71.82955
2,Andorra,12.332149,82.65301
3,Angola,5.660368,51.74093
4,Antigua and Barbuda,4.879627,75.37800


In [17]:
#check for data within the tables
pd.read_sql_query('select * from population', con=engine).head()

,country,population,gdp
0,Afghanistan,31056997,700.0
1,Albania,3581655,4500.0
2,Algeria,32930091,6000.0
3,American Samoa,57794,8000.0
4,Andorra,71201,19000.0


In [23]:
# Merging data with alchol and population df
merge_table = pd.merge(alcohol_transformed,population_transformed, on = "country")
merge_table

,country,alcohol_consumption,life_expectancy,population,gdp
0,Albania,4.879627,74.13000,3581655,4500.0
1,Algeria,0.692020,71.82955,32930091,6000.0
2,Andorra,12.332149,82.65301,71201,19000.0
3,Angola,5.660368,51.74093,12127071,1900.0
4,Argentina,7.789660,76.31537,39921833,11200.0
...,...,...,...,...,...
144,Uzbekistan,2.377709,69.20962,27307134,1700.0
145,Vanuatu,0.887205,72.28570,208869,2900.0
146,Yemen,0.106465,64.25762,21456188,800.0
147,Zambia,0.975925,58.46280,11502010,800.0


In [24]:
# Merging Data to CSV
merge_table.to_csv('merged.csv')

In [25]:
# Count merged table
merge_table.count()

country                149
alcohol_consumption    149
life_expectancy        149
population             149
gdp                    149
dtype: int64